In [1]:
import time
start_time = time.time()
import numpy as np
import os
import SimpleITK as sitk
import json
import ProstateLesionDetectionUtils
from scipy import ndimage
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)): 
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open("ProstateNetData/Datasets.json", 'r') as f:
    datasets = json.load(f)
prostatenet = "ProstateNetData/Dataset011_ProstateNet"
with open(r"folds.json", 'r') as f:
    ValidationFolds = json.load(f)
datasets
for dataset in datasets.keys():
    for pat in datasets[dataset].keys():
        for seq in datasets[dataset][pat].keys():
            x = datasets[dataset][pat][seq]
            x = x.replace("\\","/")
            datasets[dataset][pat][seq] = x

2023-09-06 10:44:06.459867: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 10:44:07.274449: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/medadmin/anaconda3/envs/tf12/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
loader = ProstateLesionDetectionUtils.DatasetsLoaders.Nifti_Loader.LoadSitkImages(ptdc = datasets["ProstateNet"], folder = prostatenet)
patients = loader.load_sitkobj()
patients_test = {key:patients[key] for key in patients.keys() if key in ValidationFolds["Fold 0"]}

In [ ]:
model_names = ['Unet', 'Attention_Unet', 'Vnet', 'USE_net', 'Dense2_Unet', 'Trans_Unet']
base_path = "EXPERIMENTS"
fold = "fold_0"
weights = "Final_2048.tf"
weights_check = "Checkpoint_2048.tf"


for md_nm in model_names:
    
    path_parent = [base_path, md_nm, fold]
    paths = [base_path, md_nm, fold,"Models", weights] # chains the paths
    model = tf.keras.models.load_model(os.path.join(*paths),compile = False)
    print(model.count_params())

    eval_params = {
        "MODEL": model,
        "EVALUATION_DATA": patients_test,# Needs to be a dictionary {PatName:{T2:arr, ADC:arr, DWI:arr, "Lesion":arr}}
        "SAVE_FOLDER":os.path.join(*path_parent) ,
    }

    eval = ProstateLesionDetectionUtils.Evaluation.Evaluation.Evaluation(eval_params)
    eval.predict()
    #eval.calculate_detorient_metric()
    eval.save_gt()
    eval.save_predictions()
    eval.save_predictions_binary()
    metrics = eval.get_metrics()
    preds = eval.get_predictions()
    
    for key,value in metrics.items():
        metrics[key][0].update({"Model": md_nm, "Number of Parameters":[model.count_params()]})

    path_parent.append(f"metrics_final_{paths[-1][:-3]}.json")
    
    with open(os.path.join(*path_parent, "Metrics_Json"), "w") as f:
        json.dump(metrics, f, cls = NumpyEncoder, indent=4)